<a href="https://colab.research.google.com/github/AthiraBabu2000/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
!pip install tflearn
!pip install flask-ngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
print("Processing the Intents.....")
with open('/content/drive/MyDrive/dataset/intents.json')as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [ ]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
331 documents
86 classes ['ai', 'architecture', 'art', 'astrology', 'beauty', 'beer', 'books', 'cars', 'cocktails', 'coffee', 'communication', 'computer_science', 'cooking', 'creativity', 'cs_community', 'cs_jobs', 'customer service', 'cyber_security', 'dancing', 'data_science', 'dating', 'education', 'entrepreneurship', 'environment', 'family', 'fashion', 'finance', 'fitness', 'food', 'gaming', 'gardening', 'goal-setting', 'goodbye', 'greeting', 'happiness', 'health', 'hiking', 'history', 'hobbies', 'hours', 'innovation', 'jokes', 'language', 'leadership', 'learning_cs', 'love', 'management', 'marketing', 'marriage', 'meditation', 'mopeds', 'motivation', 'movie', 'movies', 'music', 'nature', 'opentoday', 'parenting', 'payments', 'pets', 'philosophy', 'photography', 'pizza', 'politics', 'productivity', 'psychology', 'relationships', 'religion', 'rental', 'restaurant', 'sales', 'self-improvement', 'shoes', 'shopping', 'social media', 'sp

In [ ]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [ ]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
tf.compat.v1.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [ ]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Training....


In [ ]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [ ]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')

Training Step: 41999  | total loss: 0.00824 | time: 0.221s
| Adam | epoch: 1000 | loss: 0.00824 - acc: 0.9992 -- iter: 328/331
Training Step: 42000  | total loss: 0.00742 | time: 0.226s
| Adam | epoch: 1000 | loss: 0.00742 - acc: 0.9993 -- iter: 331/331
--
Saving the Model.......


In [ ]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [ ]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('/content/drive/MyDrive/dataset/intents.json')as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......


In [ ]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:

while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- hi
Good to see you again
You- what the weather like
It's sunny and warm today!
You- tell me a joke
Why don't scientists trust atoms? Because they make up everything!
You- do you like to travel
My favorite travel destination is Paris, it's such a beautiful city
You- what are your hobbies
In my free time, I like to practice yoga, watch movies, and cook new recipes.
You- do you like to read
My favorite book is '1984' by George Orwell
You- do you like technology
My favorite productivity app is Todoist.
You- do you speak any other language
I learned English in school and by watching TV shows and movies in English.
You- do you have any political opinion
I try to stay neutral when it comes to politics, but it's hard not to have an opinion these days.
You- how do you manage time effectively
To prioritize tasks, I use the Eisenhower Matrix. It helps me identify what's important and what's urgent.
You- what do you think of data science
I've dabbled in data science a bit, but I wouldn't call

KeyboardInterrupt: ignored